# SVM

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
from cvxopt import matrix
from cvxopt import solvers
import math
from scipy.spatial.distance import cdist
import time

In [2]:
def ReadInput():
    df = pd.read_csv('./fashion_mnist/train.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==4 ,data[:,784]==5)
    X_train = data[index][:,:-1]
    Y_train = np.array([1 if i==4 else -1 for i in data[index][:,-1]]).reshape((-1,1))


    df = pd.read_csv('./fashion_mnist/test.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==4 ,data[:,784]==5)
    X_test = data[index][:,:-1]
    Y_test = np.array([1 if i==4 else -1 for i in data[index][:,-1]]).reshape((-1,1))


    df = pd.read_csv('./fashion_mnist/val.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==4 ,data[:,784]==5)
    X_val = data[index][:,:-1]
    Y_val = np.array([1 if i==4 else -1 for i in data[index][:,-1]]).reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, X_val, Y_val


# PART 1-A 

In [9]:
def LinearSVM_fit(X,Y):
#     X = X_train/255
#     Y = Y_train
    G1 = np.diag(np.array([1]*Y.shape[0]))
    G2 = np.diag(np.array([-1]*Y.shape[0]))
    G = matrix(np.append(G1,G2,axis=0),tc='d')
    c = 1
    H = matrix(np.append(np.array([c]*Y.shape[0]),np.array([0]*Y.shape[0]),axis = 0),tc='d')
    P = matrix(np.dot(X,X.T)*np.dot(Y,Y.T),tc='d')
    Q = matrix(np.array([-1]*Y.shape[0]).T,tc='d')
    A = matrix(Y.T,tc='d')
    B = matrix(np.array(0).reshape((1,1)),tc='d')

    sol = solvers.qp(P,Q,G,H,A,B)

    alpha=np.array(sol['x'])
    
    #calculating the support vectors given a threshold
    threshold = 1e-10
    data = np.append(X,Y,axis = 1)
    data_with_alpha = np.append(data,alpha,axis = 1)
    
    supportVectors = data_with_alpha[np.where(data_with_alpha[:,-1] >= threshold)]
    print("Total supports vectors")
    print(supportVectors.shape)
    print('\n')
    # w = SUM( x*y*alpha )
    w = np.sum(supportVectors[:,:-2]*supportVectors[:,-2].reshape((-1,1))*supportVectors[:,-1].reshape((-1,1)),axis=0).reshape((-1,1))

#     pos_X = supportVectors[np.where(supportVectors[:,-2]==1)][:,:-2]
#     neg_X = supportVectors[np.where(supportVectors[:,-2]==-1)][:,:-2]
    XY = np.append(X,Y,axis=1)
    pos_X = XY[np.where(XY[:,-1]==1)][:,:-1]
    neg_X = XY[np.where(XY[:,-1]==-1)][:,:-1]
                
    print("aplha",alpha.max())
    print("aplha",alpha.min())
    b = -(np.dot(pos_X,w).min() + np.dot(neg_X,w).max())/2
    return w,b,supportVectors

In [10]:
def LinearSVM_predict(X,Y,w,b):
    pred = []
    for x in X:
        if (np.dot(w.T,x)+b) > 0:
            pred.append(1)
        else:
            pred.append(-1)

    count =0

    for i in range(len(pred)):
        if pred[i] == Y[i]:
            count+=1
    
    print("Accurarcy is ",count*100/Y.shape[0]," %") 

<font color=red>DATA INPUT</font>

In [11]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = ReadInput()

<font color=red>LEARNING</font>

In [33]:

w,b,supportVectors = LinearSVM_fit(X_train/255,Y_train)


     pcost       dcost       gap    pres   dres
 0: -1.8144e+02 -7.2340e+03  4e+04  2e+00  1e-12
 1: -1.0203e+02 -3.4492e+03  6e+03  3e-01  1e-12
 2: -3.3534e+01 -9.1974e+02  2e+03  7e-02  6e-13
 3: -1.1925e+01 -4.1859e+02  7e+02  2e-02  3e-13
 4: -3.6847e+00 -1.7770e+02  3e+02  8e-03  1e-13
 5: -1.2594e+00 -5.9907e+01  9e+01  2e-03  5e-14
 6: -6.7211e-01 -2.5705e+01  4e+01  9e-04  3e-14
 7: -4.1327e-01 -1.1925e+01  2e+01  3e-04  2e-14
 8: -5.1263e-01 -5.5495e+00  6e+00  1e-04  1e-14
 9: -5.7700e-01 -3.1645e+00  3e+00  4e-05  1e-14
10: -7.0634e-01 -2.0084e+00  1e+00  3e-06  2e-14
11: -9.3593e-01 -1.4177e+00  5e-01  7e-07  2e-14
12: -1.0291e+00 -1.2146e+00  2e-01  4e-16  2e-14
13: -1.0863e+00 -1.1319e+00  5e-02  4e-16  2e-14
14: -1.1048e+00 -1.1100e+00  5e-03  5e-16  2e-14
15: -1.1073e+00 -1.1074e+00  1e-04  5e-16  2e-14
16: -1.1073e+00 -1.1073e+00  1e-06  6e-16  2e-14
17: -1.1073e+00 -1.1073e+00  1e-08  5e-16  2e-14
Optimal solution found.
Total supports vectors
(80, 786)


aplha 0.252

<font color=red>PREDICTING</font>

In [13]:
LinearSVM_predict(X_test/255,Y_test,w,b)
LinearSVM_predict(X_val/255,Y_val,w,b)
LinearSVM_predict(X_train/255,Y_train,w,b)

Accurarcy is  99.8  %
Accurarcy is  99.6  %
Accurarcy is  100.0  %


# PART 1-B

In [16]:
def GaussianKernel(x,z,gamma):
    return np.exp(-gamma*np.sum((x-z)**2))

In [17]:
from tqdm import tqdm
def GausianSVM_fit(X,Y):
    G1 = np.diag(np.array([1]*Y.shape[0]))
    G2 = np.diag(np.array([-1]*Y.shape[0]))
    G = matrix(np.append(G1,G2,axis=0),tc='d')
    c = 1
    H = matrix(np.append(np.array([c]*Y.shape[0]),np.array([0]*Y.shape[0]),axis = 0),tc='d')
    Q = matrix(np.array([-1]*Y.shape[0]).T,tc='d')
    A = matrix(Y.T,tc='d')
    B = matrix(np.array(0).reshape((1,1)),tc='d')
    IP = []
    gamma = 0.05
    L2norm= cdist(X,X,'euclidean')
    IP = np.exp(-(L2norm**2)*gamma)
#     for i in tqdm(range(X.shape[0])):
#         L =[]
#         for j in range(X.shape[0]):
#             L.append(GaussianKernel(X[i,:],X[j,:],gamma))
#         IP.append(L)
#     IP = np.array(IP).reshape((X.shape[0],X.shape[0]))
#     print(IP.shape)
    P = matrix(IP*np.dot(Y,Y.T),tc='d')
    
    #-----------------------------------
    sol = solvers.qp(P,Q,G,H,A,B)
    alpha=np.array(sol['x'])
    #-----------------------------------
    
    #calculating the support vectors given a threshold
    threshold = 1e-6
    data = np.append(X,Y,axis = 1)
    data_with_alpha = np.append(data,alpha,axis = 1)

    supportVectors = data_with_alpha[np.where(data_with_alpha[:,-1] >= threshold)]
    print("Total supports vectors")
    print(supportVectors.shape)
    print('\n')

    XY = np.append(X,Y,axis=1)
    pos_X = XY[np.where(XY[:,-1]==1)][:,:-1]
    neg_X = XY[np.where(XY[:,-1]==-1)][:,:-1]

    #b
    b1 = math.inf
    for i in tqdm(range(pos_X.shape[0])):
        x = 0
        for j in range(supportVectors.shape[0]):
            alpha_i = supportVectors[j,-1]
            y_i = supportVectors[j,-2]
            x+= alpha_i*y_i*GaussianKernel(supportVectors[j,:-2],pos_X[i],gamma)
        b1 = min(b1,x)

    b2 = -math.inf
    for i in tqdm(range(neg_X.shape[0])):
        x = 0
        for j in range(supportVectors.shape[0]):
            alpha_i = supportVectors[j,-1]
            y_i = supportVectors[j,-2]
            x+= alpha_i*y_i*GaussianKernel(supportVectors[j,:-2],neg_X[i],gamma)
        b2 = max(b2,x)

    b = -(b1+b2)/2
    return b,supportVectors

In [18]:
#predict 
def GausianSVM_predict(X,Y,b,supportVectors):
    gamma = 0.05

    predict = []

    for i in range(X.shape[0]):
        w=0
        for j in range(supportVectors.shape[0]):
            alpha_i = supportVectors[j,-1]
            y_i = supportVectors[j,-2]
            w += alpha_i*y_i*GaussianKernel(supportVectors[j,:-2],X[i,:],gamma)
#             print((alpha_i*y_i))
        if w+b >0:
            predict.append(1)
        else:
            predict.append(-1)
    count = 0  
    for i in range(X.shape[0]):
        if predict[i] == Y[i]:
            count+=1
    print((count*100)/X.shape[0])



In [19]:
b,supportVectors = GausianSVM_fit(X_train/255,Y_train)

     pcost       dcost       gap    pres   dres
 0: -1.5598e+02 -6.9927e+03  3e+04  2e+00  1e-15
 1: -8.8657e+01 -3.4857e+03  6e+03  2e-01  1e-15
 2: -5.9446e+01 -7.3609e+02  8e+02  2e-02  2e-15
 3: -1.0523e+02 -2.6151e+02  2e+02  4e-03  2e-15
 4: -1.2171e+02 -1.8069e+02  6e+01  1e-03  1e-15
 5: -1.2850e+02 -1.5673e+02  3e+01  8e-15  1e-15
 6: -1.3314e+02 -1.4204e+02  9e+00  1e-14  1e-15
 7: -1.3446e+02 -1.3849e+02  4e+00  6e-15  1e-15
 8: -1.3538e+02 -1.3651e+02  1e+00  3e-15  1e-15
 9: -1.3573e+02 -1.3581e+02  8e-02  1e-14  1e-15
10: -1.3576e+02 -1.3576e+02  2e-03  2e-14  1e-15


  0%|          | 6/2250 [00:00<00:42, 53.20it/s]

11: -1.3576e+02 -1.3576e+02  6e-05  3e-16  1e-15
Optimal solution found.
Total supports vectors
(1028, 786)




100%|██████████| 2250/2250 [00:33<00:00, 67.26it/s]


In [20]:
GausianSVM_predict(X_test/255,Y_test,b,supportVectors)

98.8


In [21]:
GausianSVM_predict(X_val/255,Y_val,b,supportVectors)

99.6


In [22]:
GausianSVM_predict(X_train/255,Y_train,b,supportVectors)

100.0


# PART 1-C

In [29]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
def SKlearn_SVM(X_train, Y_train, X_test, Y_test, X_val, Y_val):
    clf = LinearSVC(random_state=0, tol=1e-5)
    clf.fit(X_train/255,Y_train)

    Y_pred_test=clf.predict(X_test/255)
    Acc_test = accuracy_score(Y_test, Y_pred_test)*100
    print("Accuracy over TEST data : ",Acc_test)

    Y_pred_val=clf.predict(X_val/255)
    Acc_val = accuracy_score(Y_val, Y_pred_val)*100
    print("Accuracy over VALIDATION data : ",Acc_val)

    
        
    Y_pred_train=clf.predict(X_train/255)
    Acc_train = accuracy_score(Y_train, Y_pred_train)*100
    print("Accuracy over TEST data : ",Acc_train)


In [30]:
SKlearn_SVM(X_train, Y_train, X_test, Y_test, X_val, Y_val)

Accuracy over TEST data :  99.8
Accuracy over VALIDATION data :  99.6
Accuracy over TEST data :  100.0


/home/vivek/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
